
# IRIS VQC - Blind QUantum Computing

Este notebook executa o código de Blind Quantum Computing utilizando Qiskit,  O objetivo é demonstrar como rodar experimentos quânticos de forma segura e privada, utilizando machine learning (aplicando **QNN (EstimatorQNN)** com um **classificador VQC**) com o dataset de exemplo (Iris).

Inclui:
- Pré-processamento do Iris (binário: *Setosa vs. Versicolor*)
- Construção do circuito (Feature Map + Ansatz)
- Treinamento do VQC com **ADAM**
- Envio do circuito por época
- Métricas finais e matriz de confusão

In [95]:
# Instalar dependências
!pip install qiskit qiskit-aer qiskit-machine-learning qiskit-algorithms scikit-learn numpy


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Dreysv\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [96]:
# instalar as bibliotecas necessárias

import os
import random
import numpy as np
import collections
import matplotlib

from qiskit import QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit.primitives import Estimator
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_machine_learning.utils.loss_functions import CrossEntropyLoss
from qiskit_algorithms.optimizers import ADAM

from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import itertools

from quantumnet.components import Network, Logger


# Controle de Aleatoriedade e Reprodutibilidade
O trecho abaixo define uma semente fixa (SEED = 42) e a aplica nas bibliotecas random, numpy e torch. Isso garante que os resultados aleatórios (como inicialização de pesos e embaralhamento de dados) sejam os mesmos a cada execução do código, o que é essencial para fins de reprodutibilidade. 

Além disso, o uso de torch.use_deterministic_algorithms(True) força o PyTorch a utilizar apenas algoritmos determinísticos, evitando variações internas e tornando os experimentos mais consistentes e confiáveis.

In [97]:

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)

# Callback: envio do circuito por época

In [98]:

def enviar_circuito_por_epoca(circuito, epoch, num_qubits, circuit_depth):
    print(f"[Epoch {epoch+1}] Enviando circuito para a rede...")
    try:
        rede.application_layer.run_app(
            "AC_BQC",
            alice_id=6,
            bob_id=0,
            num_qubits=num_qubits,
            scenario=2,
            circuit_depth=circuit_depth,
            circuito=circuito
        )
        print(f"[Epoch {epoch+1}] Envio concluído.")
    except Exception as e:
        print(f"[Epoch {epoch+1}] Erro ao enviar circuito: {str(e)}")


# Inicializar a rede

In [99]:

rede = Network()
rede.set_ready_topology('grade', 8, 3, 3)
Logger.activate(Logger)
print("Rede pronta.")


2025-08-18 15:07:27,772: Host 0 inicializado com 0 qubits.
2025-08-18 15:07:27,773: Host 1 inicializado com 0 qubits.
2025-08-18 15:07:27,774: Host 2 inicializado com 0 qubits.
2025-08-18 15:07:27,774: Host 3 inicializado com 0 qubits.
2025-08-18 15:07:27,775: Host 4 inicializado com 0 qubits.
2025-08-18 15:07:27,775: Host 5 inicializado com 0 qubits.
2025-08-18 15:07:27,776: Host 6 inicializado com 0 qubits.
2025-08-18 15:07:27,776: Host 7 inicializado com 0 qubits.
2025-08-18 15:07:27,776: Host 8 inicializado com 0 qubits.
2025-08-18 15:07:27,777: Par EPR <quantumnet.objects.epr.Epr object at 0x000002DE53178690> adicionado ao canal.
2025-08-18 15:07:27,778: Par EPR <quantumnet.objects.epr.Epr object at 0x000002DE52CD1510> adicionado ao canal.
2025-08-18 15:07:27,778: Par EPR <quantumnet.objects.epr.Epr object at 0x000002DE52CBB8D0> adicionado ao canal.
2025-08-18 15:07:27,778: Par EPR <quantumnet.objects.epr.Epr object at 0x000002DE52DA1350> adicionado ao canal.
2025-08-18 15:07:27,7

Hosts inicializados
Canais inicializados
Pares EPRs adicionados
Rede pronta.


# Carrega o conjunto de dados Iris
- X → dados (características/features) 
- y → rótulos (classes/labels)ris (binário)

In [100]:

iris = load_iris()
X = iris.data
y = iris.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reduz para duas classes (binário)

In [101]:

X = X[y < 2]
y = y[y < 2]


# Divide os dados 
80% treino e 20% teste


In [102]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED
)
print("Distribuição das classes no y_train e y_test:")
print("Treino:", collections.Counter(y_train))
print("Teste:", collections.Counter(y_test))

Distribuição das classes no y_train e y_test:
Treino: Counter({np.int64(1): 42, np.int64(0): 38})
Teste: Counter({np.int64(0): 12, np.int64(1): 8})


# Cria o circuito quântico (FeatureMap + Ansatz)

In [103]:

num_qubits = 4
feature_map = ZZFeatureMap(num_qubits)
ansatz = RealAmplitudes(num_qubits, reps=1)

qc = QuantumCircuit(num_qubits)
qc.compose(feature_map, inplace=True)
qc.compose(ansatz, inplace=True)

# EstimatorQNN 

In [104]:

estimator = Estimator()

qnn = EstimatorQNN(
    circuit=qc,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    estimator=estimator,
    input_gradients=True
)


C:\Users\Dreysv\AppData\Local\Temp\ipykernel_1852\1789396687.py:1: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
C:\Users\Dreysv\AppData\Local\Temp\ipykernel_1852\1789396687.py:3: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


## 8. Treinamento

In [ ]:

print("\nTREINANDO VQC...")

vqc = NeuralNetworkClassifier(
    neural_network=qnn,
    loss=CrossEntropyLoss(),
    optimizer=ADAM(maxiter=25),
    callback=lambda weights, loss, step: enviar_circuito_por_epoca(
        circuito=feature_map.compose(ansatz.assign_parameters(weights)),
        epoch=step,
        num_qubits=num_qubits,
        circuit_depth=feature_map.compose(ansatz).depth()
    )
)

vqc.fit(X_train, y_train)
print("Treinamento concluído.")



TREINANDO VQC...


## 9. Avaliação: métricas e matriz de confusão

In [ ]:

print("\nCALCULANDO MÉTRICAS...")
y_pred = vqc.predict(X_test)

y_test_int = y_test.astype(int)
y_pred_int = y_pred.astype(int)

accuracy = accuracy_score(y_test_int, y_pred_int)
precision = precision_score(y_test_int, y_pred_int, average='weighted', zero_division=0)
recall = recall_score(y_test_int, y_pred_int, average='weighted', zero_division=0)
f1 = f1_score(y_test_int, y_pred_int, average='weighted', zero_division=0)

print("\nMÉTRICAS FINAIS:")
print(f"Acurácia : {accuracy:.4f}")
print(f"Precisão : {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")